# Introduction

For my business problem I will be examining which neighborhood in North York, ON, a borough of Toronto, would be best to open a restaurant in. I will make this on two criteria. One, judging by the other restaurants in the area and two by judging how many non-restaurant venues in the area that can bring foot traffic to the area,

# Data

I will be using the Toronto location data and the Foursquare data in order to complete the analysis.

The Toronto location data has all the boroughs, postal codes and neighborhoods for the greater Toronto area.

The Foursquare data has the name, location and venue types of all the different venues in the area.

# Methodologies

I will take the Toronto location data and the Foursquare data and combine them on one another so that we can see which venues are located in which neighborhoods. Then I will cluster similar neighborhoods based off of their venues and will then make a judgment as to which neighborhood to open the new restaurant.

Import relevant packages needed for the analysis

In [57]:
import numpy as np

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


Import and clean the Toronto data and the coordinates data.

In [58]:
wikitables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [59]:
toronto = wikitables[0]

In [60]:
toronto1 = toronto[toronto['Borough'] != 'Not assigned']

In [61]:
toronto2 = toronto1.replace('Not assigned', "Queen's Park")

In [62]:
toronto3 = toronto2.groupby(['Postcode','Borough'], sort=False).agg( ', '.join)

In [63]:
toronto3.reset_index(level=0, inplace=True)

In [64]:
toronto3.reset_index(level=0, inplace=True)

In [65]:
cols = ['Postcode', 'Borough', 'Neighbourhood']
toronto4 = toronto3[cols]

In [66]:
toronto4.shape

(103, 3)

In [67]:
coordinates = pd.read_csv('http://cocl.us/Geospatial_data')
coordinates2 = coordinates.rename({'Postal Code': 'Postcode'}, axis = 1)

Join the coordinates data and the Toronto data on each other.

In [68]:
toronto5 = pd.merge(toronto4, coordinates2, on='Postcode', how='inner')

Create Map of Toronto.

In [69]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [70]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto5['Latitude'], toronto5['Longitude'], toronto5['Borough'], toronto5['Neighbourhood']):
    label = '{}, {}'.format(toronto5, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Create credentials for Foursquare API.

In [71]:
CLIENT_ID = 'RHUZYLJ2OUBP4OCASXEPTKTGTPQ1XYYFURXKFQ2SBSZCGLNI' # your Foursquare ID
CLIENT_SECRET = '4O41F5ALI0AZCNZUBYKOXN5WHIAEBCHSYRD3XQXMTDY2YIEY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RHUZYLJ2OUBP4OCASXEPTKTGTPQ1XYYFURXKFQ2SBSZCGLNI
CLIENT_SECRET:4O41F5ALI0AZCNZUBYKOXN5WHIAEBCHSYRD3XQXMTDY2YIEY


Subset the Toronto data into just the North York data.

In [72]:
north_york_data = toronto5[toronto5['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


Create a Map of North York, ON.

In [73]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [74]:
# create map of Manhattan using latitude and longitude values
map_north_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_north_york

Get the different neighbourhoods for North York and their longitudes and latitudes

In [75]:
north_york_data.loc[0, 'Neighbourhood']

'Parkwoods'

In [76]:
neighborhood_latitude = north_york_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


Get the different venue information from the foursquare API.

In [77]:
radius = 1000
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RHUZYLJ2OUBP4OCASXEPTKTGTPQ1XYYFURXKFQ2SBSZCGLNI&client_secret=4O41F5ALI0AZCNZUBYKOXN5WHIAEBCHSYRD3XQXMTDY2YIEY&v=20180605&ll=43.7532586,-79.3296565&radius=1000&limit=100'

In [78]:
results = requests.get(url).json()

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Brookbanks Park,Park,43.751976,-79.332140
2,A&W Canada,Fast Food Restaurant,43.760643,-79.326865
3,Tim Hortons,Café,43.760668,-79.326368
4,Bruno's valu-mart,Grocery Store,43.746143,-79.324630


In [81]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

29 venues were returned by Foursquare.


In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Combine the venue data with the North York data.

In [83]:
north_york_venues = getNearbyVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights, Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park, Don Mills South
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
CFB Toronto, Downsview East
Silver Hills, York Mills
Downsview West
Downsview, North Park, Upwood Park
Humber Summit
Newtonbrook, Willowdale
Downsview Central
Bedford Park, Lawrence Manor East
Emery, Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


In [84]:
print(north_york_venues.shape)
north_york_venues.head()

(249, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [85]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,4,4,4,4,4,4
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


In [86]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 110 uniques categories.


In [87]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [88]:
north_york_onehot.shape

(249, 111)

In [89]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Fast Food Restaurant,Food & Drink Shop,Food Court,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoothie Shop,Snack Place,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.105263,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.052632,0.00,0.000000,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.052632,0.000000,0.000000,0.052632,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.00,0.000000,0.052632,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.0,0.250000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.04,0.0,0.040000,0.000000,0.0,0.000000,0.000000,0.080000,0.04,0.000000,0.00,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.080000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.040000,0.000000,0.0,0.000000,0.000000,0.0000

In [90]:
north_york_grouped.shape

(23, 111)

Get the top venues for each neighborhood.

In [91]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop  0.11
1       Fast Food Restaurant  0.05
2                Supermarket  0.05
3  Middle Eastern Restaurant  0.05
4                       Park  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0       Italian Restaurant  0.12
1              Coffee Shop  0.08
2     Fast Food Restaurant  0.08
3  Comfort Food Restaurant  0.04
4         Sushi Restaurant  0.04


----CFB Toronto, Downsview East----
           venue  freq
0        Airport  0.33
1           Park  0.33
2    Snack Place  0.33
3      Pet Store  0.00
4  Movie Theater  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2


In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sandwich Place,Supermarket,Bridal Shop,Restaurant,Pizza Place,Pharmacy,Shopping Mall,Diner,Park
1,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Coffee Shop,American Restaurant,Liquor Store,Juice Bar,Comfort Food Restaurant,Café,Butcher,Pharmacy
3,"CFB Toronto, Downsview East",Airport,Park,Snack Place,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
4,Don Mills North,Japanese Restaurant,Caribbean Restaurant,Café,Gym / Fitness Center,Basketball Court,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
5,Downsview Central,Home Service,Business Service,Baseball Field,Korean Restaurant,Women's Store,Diner,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
6,Downsview Northwest,Grocery Store,Athletics & Sports,Gym / Fitness Center,Liquor Store,Discount Store,Women's Store,Diner,Concert Hall,Construction & Landscaping,Convenience Store
7,Downsview West,Grocery Store,Convenience Store,Bank,Park,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop
8,"Downsview, North Park, Upwood Park",Construction & Landscaping,Bakery,Basketball Court,Park,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop
9,"Emery, Humberlea",Baseball Field,Women's Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega


In [94]:
# set number of clusters
kclusters = 5

north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 1, 0, 3, 1, 1, 0, 0, 2], dtype=int32)

In [95]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

north_york_merged


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Pool,Food & Drink Shop,Bus Stop,Park,Women's Store,Dessert Shop,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Intersection,Hockey Arena,Pizza Place,Portuguese Restaurant,Coffee Shop,French Restaurant,Food & Drink Shop,Fast Food Restaurant,Comfort Food Restaurant,Concert Hall
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Women's Store,Miscellaneous Shop,Arts & Crafts Store,Boutique,Coffee Shop,Event Space,Fraternity House,Furniture / Home Store,Gift Shop
3,M3B,North York,Don Mills North,43.745906,-79.352188,3.0,Japanese Restaurant,Caribbean Restaurant,Café,Gym / Fitness Center,Basketball Court,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop
4,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Metro Station,Pub,Japanese Restaurant,Park,Women's Store,Dessert Shop,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
5,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1.0,Coffee Shop,Beer Store,Gym,Sporting Goods Shop,Italian Restaurant,Restaurant,Dim Sum Restaurant,Sandwich Place,Discount Store,Bike Shop
6,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Athletics & Sports,Pool,Fast Food Restaurant,Mediterranean Restaurant,Dog Run,Women's Store,Dessert Shop,Comfort Food Restaurant,Concert Hall
7,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259,1.0,Coffee Shop,Sandwich Place,Supermarket,Bridal Shop,Restaurant,Pizza Place,Pharmacy,Shopping Mall,Diner,Park
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Sporting Goods Shop,Food Court,Women's Store,Bakery,Burger Joint
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262,1.0,Caribbean Restaurant,Massage Studio,Falafel Restaurant,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


# Results

Based off the analysis above it looks like there are 24 potential neighborhoods in the North York with a variety of different venues in each. There are a total of 5 different clusters. 0,1 and 3 have multiple different neighborhoods in them showing that they are near each other based off the clustering, while 2 and 4 only have one neighborhood in each showing they are farther apart

# Discussion

Based off the results above I would recommend opening a restaurant in "Emery, Humberlea" due to the fact that it is further away from the other clusters and doesnt appear to have many restaurants in it. Only the 4th most common venue is restaurant out of all 10. It also has other venues that would draw foot traffic including a Baseball Field and a Concert Hall. Based off this there would be plenty of customers in the area who can walk into the restaurant but there will not be much competition.

# Conclusion

This concludes the report. Any feedback or input would be greatly appreciated.